In [ ]:
import ee
import numpy as np
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize()

In [ ]:
image = ee.Image("users/ganghong/Belmanip_LAI_Site1/20190402T165849_20190402T171115_T15SVV_LAI")
tiles_Bel = ee.FeatureCollection("users/ganghong/TilesSampling_Belmanip_Sub")
aoi= ee.FeatureCollection(ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(0))).geometry()
print (aoi.bounds().getInfo())
tilename=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(0)).get('MGRS_TILE')
print (tilename.getInfo())

In [34]:
dataMODIS2 =ee.ImageCollection('MODIS/006/MCD15A3H')\
                      .filterBounds(aoi)\
                      .filterDate('2019-07-01', '2019-09-01')\
                      .select(['Lai', 'Fpar','FparLai_QC'])
coarsecrs=dataMODIS2.first().projection()
 ### coarse resolution reducers  
reducers = ee.Reducer.mean().combine(
      reducer2= ee.Reducer.stdDev(),
      sharedInputs= True
    ).combine(
      reducer2= ee.Reducer.count(),
      sharedInputs= True
    )

coarseS2=image.select(['estimateLAI','errorLAI']).reproject(image.projection(), None, 20).reduceResolution(
               reducer= reducers,
               maxPixels= 65535,
               bestEffort= True
               ).reproject(
                   crs= coarsecrs,
                   scale= 500
                 )
print (coarseS2.projection().getInfo())
print (coarseS2.getInfo())


#Extract an image
#TERRA_list = coarseS2.toList(coarseS2.size())
#Terra_img = ee.Image(TERRA_list.get(1))
coarse_S2 = ee.Image(coarseS2)
coarsecrs=coarse_S2.projection()
coarsescale=coarse_S2.projection().nominalScale()
print (coarsecrs.getInfo())
print (coarsescale.getInfo())
img = coarse_S2.select('estimateLAI_mean').unmask()

# Get 2-d pixel array for AOI - returns feature with 2-D pixel array as property per band.
band_arrs = img.sampleRectangle(region=aoi.bounds())
#print (band_arrs.getInfo())
band_arr_LAI = band_arrs.get('estimateLAI_mean')
np_arr_LAI = np.array(band_arr_LAI.getInfo())
print (np_arr_LAI)
print (np_arr_LAI.shape)
print (aoi.projection().getInfo())
# # Get individual band arrays.
# band_arr_b4 = band_arrs.get('B4')
# band_arr_b5 = band_arrs.get('B5')
# band_arr_b6 = band_arrs.get('B6')

# # Transfer the arrays from server to client and cast as np array.
# np_arr_b4 = np.array(band_arr_b4.getInfo())
# np_arr_b5 = np.array(band_arr_b5.getInfo())
# np_arr_b6 = np.array(band_arr_b6.getInfo())
# print(np_arr_b4.shape)
# print(np_arr_b5.shape)
# print(np_arr_b6.shape)

# # Expand the dimensions of the images so they can be concatenated into 3-D.
# np_arr_b4 = np.expand_dims(np_arr_b4, 2)
# np_arr_b5 = np.expand_dims(np_arr_b5, 2)
# np_arr_b6 = np.expand_dims(np_arr_b6, 2)
# print(np_arr_b4.shape)
# print(np_arr_b5.shape)
# print(np_arr_b6.shape)

# # Stack the individual bands to make a 3-D array.
# rgb_img = np.concatenate((np_arr_b6, np_arr_b5, np_arr_b4), 2)
# print(rgb_img.shape)

# # Scale the data to [0, 255] to show as an RGB image.
# rgb_img_test = (255*((rgb_img - 100)/3500)).astype('uint8')
plt.imshow(np_arr_LAI)
plt.show()

{'type': 'Projection', 'crs': 'SR-ORG:6974', 'transform': [500.00000000053956, 0, -20015109.354, 0, 499.9999999994604, 10007554.677]}
{'type': 'Image', 'bands': [{'id': 'estimateLAI_mean', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'SR-ORG:6974', 'crs_transform': [500.00000000053956, 0, -20015109.354, 0, 499.9999999994604, 10007554.677]}, {'id': 'estimateLAI_stdDev', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'SR-ORG:6974', 'crs_transform': [500.00000000053956, 0, -20015109.354, 0, 499.9999999994604, 10007554.677]}, {'id': 'estimateLAI_count', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 4294967295}, 'crs': 'SR-ORG:6974', 'crs_transform': [500.00000000053956, 0, -20015109.354, 0, 499.9999999994604, 10007554.677]}, {'id': 'errorLAI_mean', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'SR-ORG:6974', 'crs_transform': [500.00000000053956, 0, -20015109

EEException: Reprojection output too large (19842x5585 pixels).